In [8]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.ensemble import AdaBoostRegressor, RandomForestRegressor, GradientBoostingRegressor, VotingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from scipy.stats import hmean, gmean
import time
import psutil
import io
import requests
import tensorflow as tf

# Set a random seed for reproducibility
np.random.seed(42)

# Load the data
# data = pd.read_csv('/content/SN_m_tot_V2.0.csv', delimiter=';', header=None)
# url = 'https://www.sidc.be/SILSO/INFO/snmtotcsv.php'
url = 'https://www.sidc.be/SILSO/INFO/snmstotcsv.php'
data = pd.read_csv(url, delimiter=';', header=None)
data.columns = ['Year', 'Month', 'Date', 'Monthly Mean Total Sunspot Number', 'Uncertainty', 'Observations', 'Definitive/Provisional']

# Select the 'Monthly Mean Total Sunspot Number' column as the target variable
target = data['Monthly Mean Total Sunspot Number'].values.reshape(-1, 1)

# Normalize the target variable
#scaler = MinMaxScaler(feature_range=(0, 1))
#target_scaled = scaler.fit_transform(target)

# Split the data into training and testing sets
train_size = 720  # Number of months for training
test_size = 72  # Number of months for testing

train_data = target[:train_size]
test_data = target[train_size:train_size+test_size]


# Split into input and output variables
X_train = train_data[:-1]
y_train = train_data[1:]
X_test = test_data[:-1]
y_test = test_data[1:]

if X_test.shape[0] == 0:
    X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, shuffle=False)

# Define the regression models to be evaluated
models = [
    LinearRegression(),
    SVR(),
    AdaBoostRegressor(),
    RandomForestRegressor(),
    GradientBoostingRegressor()
]

# Train and evaluate each model without hyperparameter tuning
best_model_predictions = []
best_model_smapes = []
best_model_mses = []
best_model_rmses = []
best_model_maes = []
best_model_r2s = []
training_times = []
cpu_usages = []
memory_usages = []

for model in models:
    # Fit the model to the training data
    start_time = time.time()
    model.fit(X_train, y_train.ravel())
    training_time = time.time() - start_time

    # Get the predictions
    predictions = model.predict(X_test).ravel()

    # Append the predictions to the list
    best_model_predictions.append(predictions)

    # Compute evaluation metrics for the model
    mse = mean_squared_error(y_test, predictions)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_test, predictions)
    r2 = r2_score(y_test, predictions)
    smape_val = smape(y_test, predictions)

    best_model_smapes.append(smape_val)
    best_model_mses.append(mse)
    best_model_rmses.append(rmse)
    best_model_maes.append(mae)
    best_model_r2s.append(r2)

    # Print evaluation metrics
    model_name = model.__class__.__name__
    cpu_usage = psutil.cpu_percent()
    print(f"{model_name} RMSE: {rmse:.5f}")
    print(f"{model_name} MAE: {mae:.5f}")
    print(f"{model_name} R-squared: {r2:.5f}")
    print(f"{model_name} SMAPE: {smape_val:.5f}")
    print(f"{model_name} Training Time: {training_time:.5f} seconds")
    print(f"CPU Usage: {cpu_usage:.1f} MHz")
    print(f"Memory Used: {psutil.virtual_memory().used / 1024 / 1024} MB")
    print()

    # Append training time, CPU usage, and memory usage to lists
    training_times.append(training_time)
    cpu_usages.append(cpu_usage)
    memory_usages.append(psutil.virtual_memory().used / 1024 / 1024)

# Combine the predictions of the best models
combined_predictions = np.mean(best_model_predictions, axis=0)
median_predictions = np.median(best_model_predictions, axis=0)

# Compute evaluation metrics for the combined model
combined_mse = mean_squared_error(y_test, combined_predictions)
combined_rmse = np.sqrt(combined_mse)
combined_mae = mean_absolute_error(y_test, combined_predictions)
combined_r2 = r2_score(y_test, combined_predictions)
combined_smape = smape(y_test, combined_predictions)

# Compute evaluation metrics for the median model
median_mse = mean_squared_error(y_test, median_predictions)
median_rmse = np.sqrt(median_mse)
median_mae = mean_absolute_error(y_test, median_predictions)
median_r2 = r2_score(y_test, median_predictions)
median_smape = smape(y_test, median_predictions)

# Compute harmonic mean and geometric mean of the predictions
harmonic_mean_predictions = hmean(best_model_predictions)
geometric_mean_predictions = gmean(best_model_predictions)

# Compute evaluation metrics for harmonic mean predictions
harmonic_mean_mse = mean_squared_error(y_test, harmonic_mean_predictions)
harmonic_mean_rmse = np.sqrt(harmonic_mean_mse)
harmonic_mean_mae = mean_absolute_error(y_test, harmonic_mean_predictions)
harmonic_mean_r2 = r2_score(y_test, harmonic_mean_predictions)
harmonic_mean_smape = smape(y_test, harmonic_mean_predictions)

# Compute evaluation metrics for geometric mean predictions
geometric_mean_mse = mean_squared_error(y_test, geometric_mean_predictions)
geometric_mean_rmse = np.sqrt(geometric_mean_mse)
geometric_mean_mae = mean_absolute_error(y_test, geometric_mean_predictions)
geometric_mean_r2 = r2_score(y_test, geometric_mean_predictions)
geometric_mean_smape = smape(y_test, geometric_mean_predictions)

# Prepare the data for the evaluation metrics table
metrics_data = {
    'Model': [model.__class__.__name__ for model in models] + ['Combined Model', 'Median Model', 'Harmonic Mean Model', 'Geometric Mean Model'],
    'RMSE': best_model_rmses + [combined_rmse, median_rmse, harmonic_mean_rmse, geometric_mean_rmse],
    'MAE': best_model_maes + [combined_mae, median_mae, harmonic_mean_mae, geometric_mean_mae],
    'R-squared': best_model_r2s + [combined_r2, median_r2, harmonic_mean_r2, geometric_mean_r2],
    'SMAPE': best_model_smapes + [combined_smape, median_smape, harmonic_mean_smape, geometric_mean_smape],
    'Training Time (seconds)': training_times + [None] * 4,
    'CPU Usage (MHz)': cpu_usages + [None] * 4,
    'Memory Used (MB)': memory_usages + [None] * 4
}

# Save the evaluation metrics to a CSV file
metrics_df = pd.DataFrame(metrics_data)
metrics_df.to_csv('Reg_metrics_test1.csv', index=False)


LinearRegression RMSE: 1.00037
LinearRegression MAE: 0.78020
LinearRegression R-squared: 0.98988
LinearRegression SMAPE: 1.18627
LinearRegression Training Time: 0.06880 seconds
CPU Usage: 15.3 MHz
Memory Used: 13015.95703125 MB

SVR RMSE: 13.51717
SVR MAE: 11.22388
SVR R-squared: -0.84728
SVR SMAPE: 1.03762
SVR Training Time: 0.09827 seconds
CPU Usage: 77.9 MHz
Memory Used: 13016.34765625 MB

AdaBoostRegressor RMSE: 28.25056
AdaBoostRegressor MAE: 21.87484
AdaBoostRegressor R-squared: -7.06893
AdaBoostRegressor SMAPE: 1.16240
AdaBoostRegressor Training Time: 0.03089 seconds
CPU Usage: 92.3 MHz
Memory Used: 13019.69921875 MB

RandomForestRegressor RMSE: 12.19346
RandomForestRegressor MAE: 7.49687
RandomForestRegressor R-squared: -0.50320
RandomForestRegressor SMAPE: 1.03598
RandomForestRegressor Training Time: 0.69040 seconds
CPU Usage: 37.8 MHz
Memory Used: 13030.484375 MB

GradientBoostingRegressor RMSE: 11.30278
GradientBoostingRegressor MAE: 7.01908
GradientBoostingRegressor R-squar

In [19]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Bidirectional, GRU
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from scipy.stats import hmean, gmean
import time
import psutil
import io
import requests

# Set a random seed for reproducibility
np.random.seed(42)

# Load the data
# data = pd.read_csv('/content/SN_m_tot_V2.0.csv', delimiter=';', header=None)
# url = 'https://www.sidc.be/SILSO/INFO/snmtotcsv.php'
url = 'https://www.sidc.be/SILSO/INFO/snmstotcsv.php'
data = pd.read_csv(url, delimiter=';', header=None)
data.columns = ['Year', 'Month', 'Date', 'Monthly Mean Total Sunspot Number', 'Uncertainty', 'Observations', 'Definitive/Provisional']

# Select the 'Monthly Mean Total Sunspot Number' column as the target variable
target = data['Monthly Mean Total Sunspot Number'].values.reshape(-1, 1)

# Normalize the target variable
#scaler = MinMaxScaler(feature_range=(0, 1))
#target_scaled = scaler.fit_transform(target)

# Split the data into training and testing sets
train_size = 720  # Number of months for training
test_size = 72  # Number of months for testing

train_data = target[:train_size]
test_data = target[train_size:train_size+test_size]

# Split into input and output variables
X_train = train_data[:-1]
y_train = train_data[1:]
X_test = test_data[:-1]
y_test = test_data[1:]

# Define the deep learning models to be evaluated
def create_lstm_model(units):
    model = Sequential()
    model.add(LSTM(units, input_shape=(1, X_train.shape[1])))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

def create_stacked_lstm_model(units):
    model = Sequential()
    model.add(LSTM(units, input_shape=(1, X_train.shape[1]), return_sequences=True))
    model.add(LSTM(units))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

def create_bidirectional_lstm_model(units):
    model = Sequential()
    model.add(Bidirectional(LSTM(units), input_shape=(1, X_train.shape[1])))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

def create_gru_model(units):
    model = Sequential()
    model.add(GRU(units, input_shape=(1, X_train.shape[1])))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

# Compute SMAPE (Symmetric Mean Absolute Percentage Error)
def smape(y_true, y_pred):
    return 200 * np.mean(np.abs(y_true - y_pred) / (np.abs(y_true) + np.abs(y_pred)))

# Train and evaluate each deep learning model
models = {
    'LSTM': create_lstm_model(128),
    'Stacked LSTM': create_stacked_lstm_model(128),
    'Bidirectional LSTM': create_bidirectional_lstm_model(128),
    'GRU': create_gru_model(128)
}

# Create lists to store evaluation metrics
model_data = {'True Value': y_test.ravel()}

rmse_list = []
mae_list = []
r2_list = []
smape_list = []
training_times = []
cpu_usages = []
memory_usages = []

for model_name, model in models.items():
    start_time = time.time()
    history = model.fit(X_train.reshape((X_train.shape[0], 1, X_train.shape[1])),
                        y_train,
                        epochs=100,
                        batch_size=8,
                        verbose=0)
    training_time = time.time() - start_time

    predictions = model.predict(X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))).ravel()
    mse = mean_squared_error(y_test, predictions)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_test, predictions)
    r2 = r2_score(y_test, predictions)
    smape_val = smape(y_test, predictions)

    # Store the predictions in the dictionary
    model_data[model_name] = predictions

    # Append evaluation metrics to the lists
    rmse_list.append(rmse)
    mae_list.append(mae)
    r2_list.append(r2)
    smape_list.append(smape_val)
    training_times.append(training_time)
    cpu_usages.append(psutil.cpu_percent())
    memory_usages.append(psutil.virtual_memory().used / 1024 / 1024)

    print(f"Model: {model_name}")
    print(f"RMSE: {rmse:.5f}")
    print(f"MAE: {mae:.5f}")
    print(f"R-squared: {r2:.5f}")
    print(f"SMAPE: {smape_val:.5f}")
    print(f"Training Time: {training_time:.5f} seconds")
    print(f"CPU Usage: {psutil.cpu_percent()} MHz")
    print(f"Memory Used: {psutil.virtual_memory().used / 1024 / 1024} MB")
    print()

# Combine the predictions of the models
combined_predictions = np.mean(list(model_data.values()), axis=0)
median_predictions = np.median(list(model_data.values()), axis=0)
harmonic_mean_predictions = hmean(list(model_data.values()))
geometric_mean_predictions = gmean(list(model_data.values()))

# Compute evaluation metrics for the combined model
combined_mse = mean_squared_error(y_test, combined_predictions)
combined_rmse = np.sqrt(combined_mse)
combined_mae = mean_absolute_error(y_test, combined_predictions)
combined_r2 = r2_score(y_test, combined_predictions)
combined_smape = smape(y_test, combined_predictions)

# Compute evaluation metrics for the median model
median_mse = mean_squared_error(y_test, median_predictions)
median_rmse = np.sqrt(median_mse)
median_mae = mean_absolute_error(y_test, median_predictions)
median_r2 = r2_score(y_test, median_predictions)
median_smape = smape(y_test, median_predictions)

# Compute evaluation metrics for harmonic mean predictions
harmonic_mean_mse = mean_squared_error(y_test, harmonic_mean_predictions)
harmonic_mean_rmse = np.sqrt(harmonic_mean_mse)
harmonic_mean_mae = mean_absolute_error(y_test, harmonic_mean_predictions)
harmonic_mean_r2 = r2_score(y_test, harmonic_mean_predictions)
harmonic_mean_smape = smape(y_test, harmonic_mean_predictions)

# Compute evaluation metrics for geometric mean predictions
geometric_mean_mse = mean_squared_error(y_test, geometric_mean_predictions)
geometric_mean_rmse = np.sqrt(geometric_mean_mse)
geometric_mean_mae = mean_absolute_error(y_test, geometric_mean_predictions)
geometric_mean_r2 = r2_score(y_test, geometric_mean_predictions)
geometric_mean_smape = smape(y_test, geometric_mean_predictions)

# Save forecasts to a CSV file
df_data = pd.DataFrame(model_data)
df_data.to_csv('DL_forecasts_test122.csv', index=False)

# Prepare the data for the evaluation metrics table
metrics_data = {
    'Model': list(models.keys()) + ['Combined Model', 'Median Model', 'Harmonic Mean Model', 'Geometric Mean Model'],
    'RMSE': rmse_list + [combined_rmse, median_rmse, harmonic_mean_rmse, geometric_mean_rmse],
    'MAE': mae_list + [combined_mae, median_mae, harmonic_mean_mae, geometric_mean_mae],
    'R-squared': r2_list + [combined_r2, median_r2, harmonic_mean_r2, geometric_mean_r2],
    'SMAPE': smape_list + [combined_smape, median_smape, harmonic_mean_smape, geometric_mean_smape],
    'Training Time (seconds)': training_times + [None] * 4,
    'CPU Usage (MHz)': cpu_usages + [None] * 4,
    'Memory Used (MB)': memory_usages + [None] * 4
}

# Save the evaluation metrics to a CSV file
metrics_df = pd.DataFrame(metrics_data)
metrics_df.to_csv('DL_metrics_test122.csv', index=False)


3/3 [==============================] - 1s 5ms/step
Model: LSTM
RMSE: 4.17160
MAE: 3.49857
R-squared: 0.82406
SMAPE: 101.41411
Training Time: 46.67141 seconds
CPU Usage: 20.0 MHz
Memory Used: 13639.12890625 MB

3/3 [==============================] - 1s 7ms/step
Model: Stacked LSTM
RMSE: 6.00223
MAE: 4.87600
R-squared: 0.63576
SMAPE: 100.85575
Training Time: 86.06820 seconds
CPU Usage: 0.0 MHz
Memory Used: 13662.46875 MB

3/3 [==============================] - 1s 10ms/step
Model: Bidirectional LSTM
RMSE: 4.95809
MAE: 3.99177
R-squared: 0.75146
SMAPE: 101.07885
Training Time: 62.46402 seconds
CPU Usage: 0.0 MHz
Memory Used: 13675.38671875 MB

3/3 [==============================] - 1s 9ms/step
Model: GRU
RMSE: 5.93466
MAE: 5.08392
R-squared: 0.64392
SMAPE: 100.32400
Training Time: 46.52151 seconds
CPU Usage: 0.0 MHz
Memory Used: 13678.203125 MB



C:\Users\Storm\anaconda3\lib\site-packages\scipy\stats\stats.py:275: RuntimeWarning: divide by zero encountered in log
  log_a = np.log(np.array(a, dtype=dtype))
C:\Users\Storm\AppData\Local\Temp/ipykernel_15340/46033814.py:76: RuntimeWarning: invalid value encountered in true_divide
  return 200 * np.mean(np.abs(y_true - y_pred) / (np.abs(y_true) + np.abs(y_pred)))
C:\Users\Storm\AppData\Local\Temp/ipykernel_15340/46033814.py:76: RuntimeWarning: invalid value encountered in true_divide
  return 200 * np.mean(np.abs(y_true - y_pred) / (np.abs(y_true) + np.abs(y_pred)))


In [22]:
import numpy as np
import pandas as pd
from scipy.stats import hmean, gmean
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Bidirectional, GRU
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import time
import psutil

# Set a random seed for reproducibility
np.random.seed(42)

# Load the data
url = 'https://www.sidc.be/SILSO/INFO/snmstotcsv.php'
data = pd.read_csv(url, delimiter=';', header=None)
data.columns = ['Year', 'Month', 'Date', 'Monthly Mean Total Sunspot Number', 'Uncertainty', 'Observations', 'Definitive/Provisional']

# Select the 'Monthly Mean Total Sunspot Number' column as the target variable
target = data['Monthly Mean Total Sunspot Number'].values.reshape(-1, 1)

# Split the data into training and testing sets
train_size = 720  # Number of months for training
test_size = 72  # Number of months for testing

train_data = target[:train_size]
test_data = target[train_size:train_size+test_size]

# Split into input and output variables
X_train = train_data[:-1]
y_train = train_data[1:]
X_test = test_data[:-1]
y_test = test_data[1:]

# Define the deep learning models to be evaluated
def create_lstm_model(units):
    model = Sequential()
    model.add(LSTM(units, input_shape=(1, X_train.shape[1])))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

def create_bidirectional_lstm_model(units):
    model = Sequential()
    model.add(Bidirectional(LSTM(units), input_shape=(1, X_train.shape[1])))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

def create_gru_model(units):
    model = Sequential()
    model.add(GRU(units, input_shape=(1, X_train.shape[1])))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

# Compute SMAPE (Symmetric Mean Absolute Percentage Error)
def smape(y_true, y_pred):
    epsilon = 1e-10
    percentage_error = 200 * np.abs(y_true - y_pred) / (np.abs(y_true) + np.abs(y_pred) + epsilon)
    return np.mean(percentage_error)

# ... (the rest of the code remains unchanged) ...


# Train and evaluate models
def train_and_evaluate_model(model, model_name, X_train, y_train, X_test, y_test):
    start_time = time.time()
    history = model.fit(X_train.reshape((X_train.shape[0], 1, X_train.shape[1])),
                        y_train,
                        epochs=100,
                        batch_size=8,
                        verbose=0)
    training_time = time.time() - start_time

    predictions = model.predict(X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))).ravel()
    mse = mean_squared_error(y_test, predictions)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_test, predictions)
    r2 = r2_score(y_test, predictions)
    smape_val = smape(y_test, predictions)

    print(f"Model: {model_name}")
    print(f"RMSE: {rmse:.5f}")
    print(f"MAE: {mae:.5f}")
    print(f"R-squared: {r2:.5f}")
    print(f"SMAPE: {smape_val:.5f}")
    print(f"Training Time: {training_time:.5f} seconds")
    print(f"CPU Usage: {psutil.cpu_percent()} MHz")
    print(f"Memory Used: {psutil.virtual_memory().used / 1024 / 1024} MB")
    print()

    return predictions, rmse, mae, r2, smape_val

# Create models
models = {
    'LSTM': create_lstm_model(128),
    'Stacked LSTM': create_stacked_lstm_model(128),
    'Bidirectional LSTM': create_bidirectional_lstm_model(128),
    'GRU': create_gru_model(128)
}

# Create a dictionary to store the forecasts and actual values for each model
model_data = {'True Value': y_test.ravel()}

# Train and evaluate each deep learning model
for model_name, model in models.items():
    predictions, rmse, mae, r2, smape_val = train_and_evaluate_model(model, model_name, X_train, y_train, X_test, y_test)
    
    # Store the predictions in the dictionary
    model_data[model_name] = predictions

# Combine the predictions of the models
combined_predictions = np.mean(list(model_data.values()), axis=0)
median_predictions = np.median(list(model_data.values()), axis=0)
harmonic_mean_predictions = hmean(list(model_data.values()))
geometric_mean_predictions = gmean(list(model_data.values()))

# Print evaluation metrics for the combined model
combined_mse = mean_squared_error(y_test, combined_predictions)
combined_rmse = np.sqrt(combined_mse)
combined_mae = mean_absolute_error(y_test, combined_predictions)
combined_r2 = r2_score(y_test, combined_predictions)
combined_smape = smape(y_test, combined_predictions)

print("Combined Model:")
print(f"RMSE: {combined_rmse:.5f}")
print(f"MAE: {combined_mae:.5f}")
print(f"R-squared: {combined_r2:.5f}")
print(f"SMAPE: {combined_smape:.5f}")
print()

# Print evaluation metrics for the median model
median_mse = mean_squared_error(y_test, median_predictions)
median_rmse = np.sqrt(median_mse)
median_mae = mean_absolute_error(y_test, median_predictions)
median_r2 = r2_score(y_test, median_predictions)
median_smape = smape(y_test, median_predictions)

print("Median Model:")
print(f"RMSE: {median_rmse:.5f}")
print(f"MAE: {median_mae:.5f}")
print(f"R-squared: {median_r2:.5f}")
print(f"SMAPE: {median_smape:.5f}")
print()

# Print evaluation metrics for harmonic mean predictions
harmonic_mean_mse = mean_squared_error(y_test, harmonic_mean_predictions)
harmonic_mean_rmse = np.sqrt(harmonic_mean_mse)
harmonic_mean_mae = mean_absolute_error(y_test, harmonic_mean_predictions)
harmonic_mean_r2 = r2_score(y_test, harmonic_mean_predictions)
harmonic_mean_smape = smape(y_test, harmonic_mean_predictions)

print("Harmonic Mean Model:")
print(f"RMSE: {harmonic_mean_rmse:.5f}")
print(f"MAE: {harmonic_mean_mae:.5f}")
print(f"R-squared: {harmonic_mean_r2:.5f}")
print(f"SMAPE: {harmonic_mean_smape:.5f}")
print()

# Print evaluation metrics for geometric mean predictions
geometric_mean_mse = mean_squared_error(y_test, geometric_mean_predictions)
geometric_mean_rmse = np.sqrt(geometric_mean_mse)
geometric_mean_mae = mean_absolute_error(y_test, geometric_mean_predictions)
geometric_mean_r2 = r2_score(y_test, geometric_mean_predictions)
geometric_mean_smape = smape(y_test, geometric_mean_predictions)

print("Geometric Mean Model:")
print(f"RMSE: {geometric_mean_rmse:.5f}")
print(f"MAE: {geometric_mean_mae:.5f}")
print(f"R-squared: {geometric_mean_r2:.5f}")
print(f"SMAPE: {geometric_mean_smape:.5f}")
print()

# Prepare the data for the evaluation metrics table
metrics_data = {
    'Model': list(models.keys()) + ['Combined Model', 'Median Model', 'Harmonic Mean Model', 'Geometric Mean Model'],
    'RMSE': list(mean_rmse.values()) + [combined_rmse, median_rmse, harmonic_mean_rmse, geometric_mean_rmse],
    'MAE': list(mean_mae.values()) + [combined_mae, median_mae, harmonic_mean_mae, geometric_mean_mae],
    'R-squared': list(mean_r2.values()) + [combined_r2, median_r2, harmonic_mean_r2, geometric_mean_r2],
    'SMAPE': list(mean_smape.values()) + [combined_smape, median_smape, harmonic_mean_smape, geometric_mean_smape],
}

# Save the evaluation metrics to a CSV file
metrics_df = pd.DataFrame(metrics_data)
metrics_df.to_csv('DL_metrics.csv', index=False)


3/3 [==============================] - 1s 4ms/step
Model: LSTM
RMSE: 4.68633
MAE: 3.88201
R-squared: 0.77796
SMAPE: 101.24834
Training Time: 49.80743 seconds
CPU Usage: 19.2 MHz
Memory Used: 13958.94140625 MB

3/3 [==============================] - 1s 0s/step
Model: Stacked LSTM
RMSE: 5.71059
MAE: 4.90557
R-squared: 0.67030
SMAPE: 100.63671
Training Time: 86.05765 seconds
CPU Usage: 43.0 MHz
Memory Used: 14068.75390625 MB

3/3 [==============================] - 1s 3ms/step
Model: Bidirectional LSTM
RMSE: 5.42839
MAE: 4.28589
R-squared: 0.70208
SMAPE: 100.93393
Training Time: 68.58537 seconds
CPU Usage: 56.9 MHz
Memory Used: 14044.6796875 MB

3/3 [==============================] - 1s 3ms/step
Model: GRU
RMSE: 6.51305
MAE: 4.91354
R-squared: 0.57113
SMAPE: 101.22677
Training Time: 48.87820 seconds
CPU Usage: 48.8 MHz
Memory Used: 14038.4921875 MB

Combined Model:
RMSE: 4.43649
MAE: 3.56793
R-squared: 0.80101
SMAPE: 100.99845

Median Model:
RMSE: 5.27169
MAE: 4.15229
R-squared: 0.71903
SM

C:\Users\Storm\anaconda3\lib\site-packages\scipy\stats\stats.py:275: RuntimeWarning: divide by zero encountered in log
  log_a = np.log(np.array(a, dtype=dtype))


NameError: name 'mean_rmse' is not defined

In [24]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Bidirectional, GRU
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from scipy.stats import hmean, gmean
import time
import psutil
import io
import requests

# Set a random seed for reproducibility
np.random.seed(42)

# Load the data
url = 'https://www.sidc.be/SILSO/INFO/snmstotcsv.php'
data = pd.read_csv(url, delimiter=';', header=None)
data.columns = ['Year', 'Month', 'Date', 'Monthly Mean Total Sunspot Number', 'Uncertainty', 'Observations', 'Definitive/Provisional']

# Select the 'Monthly Mean Total Sunspot Number' column as the target variable
target = data['Monthly Mean Total Sunspot Number'].values.reshape(-1, 1)

# Split the data into training and testing sets
train_size = 720  # Number of months for training
test_size = 72  # Number of months for testing

train_data = target[:train_size]
test_data = target[train_size:train_size+test_size]

# Split into input and output variables
X_train = train_data[:-1]
y_train = train_data[1:]
X_test = test_data[:-1]
y_test = test_data[1:]

# Create deep learning models
def create_lstm_model(units):
    model = Sequential()
    model.add(LSTM(units, input_shape=(1, X_train.shape[1])))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

def create_bidirectional_lstm_model(units):
    model = Sequential()
    model.add(Bidirectional(LSTM(units), input_shape=(1, X_train.shape[1])))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

def create_gru_model(units):
    model = Sequential()
    model.add(GRU(units, input_shape=(1, X_train.shape[1])))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

# Compute SMAPE (Symmetric Mean Absolute Percentage Error)
def smape(y_true, y_pred):
    return 200 * np.mean(np.abs(y_true - y_pred) / (np.abs(y_true) + np.abs(y_pred)))

# Train and evaluate models
def train_and_evaluate_model(model, model_name, X_train, y_train, X_test, y_test):
    start_time = time.time()
    history = model.fit(X_train.reshape((X_train.shape[0], 1, X_train.shape[1])),
                        y_train,
                        epochs=100,
                        batch_size=8,
                        verbose=0)
    training_time = time.time() - start_time

    predictions = model.predict(X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))).ravel()
    mse = mean_squared_error(y_test, predictions)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_test, predictions)
    r2 = r2_score(y_test, predictions)
    smape_val = smape(y_test, predictions)

    print(f"Model: {model_name}")
    print(f"RMSE: {rmse:.5f}")
    print(f"MAE: {mae:.5f}")
    print(f"R-squared: {r2:.5f}")
    print(f"SMAPE: {smape_val:.5f}")
    print(f"Training Time: {training_time:.5f} seconds")
    print(f"CPU Usage: {psutil.cpu_percent()} MHz")
    print(f"Memory Used: {psutil.virtual_memory().used / 1024 / 1024} MB")
    print()

    return predictions, rmse, mae, r2, smape_val

# Create models
models = {
    'LSTM': create_lstm_model(128),
    'Bidirectional LSTM': create_bidirectional_lstm_model(128),
    'GRU': create_gru_model(128)
}

# Create lists to store evaluation metrics
best_model_predictions = {}
best_model_rmses = {}
best_model_maes = {}
best_model_r2s = {}
best_model_smapes = {}

# Train and evaluate each deep learning model
for model_name, model in models.items():
    predictions, rmse, mae, r2, smape_val = train_and_evaluate_model(model, model_name, X_train, y_train, X_test, y_test)

    best_model_predictions[model_name] = predictions
    best_model_rmses[model_name] = rmse
    best_model_maes[model_name] = mae
    best_model_r2s[model_name] = r2
    best_model_smapes[model_name] = smape_val

# Combine the predictions of the best models
combined_predictions = np.mean(list(best_model_predictions.values()), axis=0)
median_predictions = np.median(list(best_model_predictions.values()), axis=0)

# Compute evaluation metrics for the combined model
combined_mse = mean_squared_error(y_test, combined_predictions)
combined_rmse = np.sqrt(combined_mse)
combined_mae = mean_absolute_error(y_test, combined_predictions)
combined_r2 = r2_score(y_test, combined_predictions)
combined_smape = smape(y_test, combined_predictions)

# Compute evaluation metrics for the median model
median_mse = mean_squared_error(y_test, median_predictions)
median_rmse = np.sqrt(median_mse)
median_mae = mean_absolute_error(y_test, median_predictions)
median_r2 = r2_score(y_test, median_predictions)
median_smape = smape(y_test, median_predictions)

# Compute harmonic mean and geometric mean of the predictions
harmonic_mean_predictions = hmean(list(best_model_predictions.values()))
geometric_mean_predictions = gmean(list(best_model_predictions.values()))

# Compute evaluation metrics for harmonic mean predictions
harmonic_mean_mse = mean_squared_error(y_test, harmonic_mean_predictions)
harmonic_mean_rmse = np.sqrt(harmonic_mean_mse)
harmonic_mean_mae = mean_absolute_error(y_test, harmonic_mean_predictions)
harmonic_mean_r2 = r2_score(y_test, harmonic_mean_predictions)
harmonic_mean_smape = smape(y_test, harmonic_mean_predictions)

# Compute evaluation metrics for geometric mean predictions
geometric_mean_mse = mean_squared_error(y_test, geometric_mean_predictions)
geometric_mean_rmse = np.sqrt(geometric_mean_mse)
geometric_mean_mae = mean_absolute_error(y_test, geometric_mean_predictions)
geometric_mean_r2 = r2_score(y_test, geometric_mean_predictions)
geometric_mean_smape = smape(y_test, geometric_mean_predictions)

# Prepare the data for the evaluation metrics table
metrics_data = {
    'Model': list(models.keys()) + ['Combined Model', 'Median Model', 'Harmonic Mean Model', 'Geometric Mean Model'],
    'RMSE': list(best_model_rmses.values()) + [combined_rmse, median_rmse, harmonic_mean_rmse, geometric_mean_rmse],
    'MAE': list(best_model_maes.values()) + [combined_mae, median_mae, harmonic_mean_mae, geometric_mean_mae],
    'R-squared': list(best_model_r2s.values()) + [combined_r2, median_r2, harmonic_mean_r2, geometric_mean_r2],
    'SMAPE': list(best_model_smapes.values()) + [combined_smape, median_smape, harmonic_mean_smape, geometric_mean_smape]
}

# Create a DataFrame with the evaluation metrics
metrics_df = pd.DataFrame(metrics_data)

# Save the evaluation metrics to a CSV file
metrics_df.to_csv('deep_learning_model_metrics.csv', index=False)


3/3 [==============================] - 1s 3ms/step
Model: LSTM
RMSE: 4.92054
MAE: 4.07627
R-squared: 0.75521
SMAPE: 100.82266
Training Time: 33.46689 seconds
CPU Usage: 15.6 MHz
Memory Used: 13896.890625 MB

3/3 [==============================] - 1s 4ms/step
Model: Bidirectional LSTM
RMSE: 5.79108
MAE: 4.49262
R-squared: 0.66094
SMAPE: 100.98292
Training Time: 46.30838 seconds
CPU Usage: 48.1 MHz
Memory Used: 13968.41015625 MB

3/3 [==============================] - 0s 3ms/step
Model: GRU
RMSE: 5.67785
MAE: 4.51462
R-squared: 0.67407
SMAPE: 101.18125
Training Time: 32.36674 seconds
CPU Usage: 39.7 MHz
Memory Used: 13980.62109375 MB



In [25]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Bidirectional, GRU
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from scipy.stats import hmean, gmean
import time
import psutil
import io
import requests

# Set a random seed for reproducibility
np.random.seed(42)

# Load the data
url = 'https://www.sidc.be/SILSO/INFO/snmstotcsv.php'
data = pd.read_csv(url, delimiter=';', header=None)
data.columns = ['Year', 'Month', 'Date', 'Monthly Mean Total Sunspot Number', 'Uncertainty', 'Observations', 'Definitive/Provisional']

# Select the 'Monthly Mean Total Sunspot Number' column as the target variable
target = data['Monthly Mean Total Sunspot Number'].values.reshape(-1, 1)

# Split the data into training and testing sets
train_size = 720  # Number of months for training
test_size = 72  # Number of months for testing

train_data = target[:train_size]
test_data = target[train_size:train_size+test_size]

# Split into input and output variables
X_train = train_data[:-1]
y_train = train_data[1:]
X_test = test_data[:-1]
y_test = test_data[1:]

# Create deep learning models
def create_lstm_model(units):
    model = Sequential()
    model.add(LSTM(units, input_shape=(1, X_train.shape[1])))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

def create_stacked_lstm_model(units):
    model = Sequential()
    model.add(LSTM(units, return_sequences=True, input_shape=(1, X_train.shape[1])))
    model.add(LSTM(units))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

def create_bidirectional_lstm_model(units):
    model = Sequential()
    model.add(Bidirectional(LSTM(units), input_shape=(1, X_train.shape[1])))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

def create_gru_model(units):
    model = Sequential()
    model.add(GRU(units, input_shape=(1, X_train.shape[1])))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

# Compute SMAPE (Symmetric Mean Absolute Percentage Error)
def smape(y_true, y_pred):
    return 200 * np.mean(np.abs(y_true - y_pred) / (np.abs(y_true) + np.abs(y_pred)))

# Train and evaluate models
def train_and_evaluate_model(model, model_name, X_train, y_train, X_test, y_test):
    start_time = time.time()
    history = model.fit(X_train.reshape((X_train.shape[0], 1, X_train.shape[1])),
                        y_train,
                        epochs=100,
                        batch_size=8,
                        verbose=0)
    training_time = time.time() - start_time

    predictions = model.predict(X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))).ravel()
    mse = mean_squared_error(y_test, predictions)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_test, predictions)
    r2 = r2_score(y_test, predictions)
    smape_val = smape(y_test, predictions)

    print(f"Model: {model_name}")
    print(f"RMSE: {rmse:.5f}")
    print(f"MAE: {mae:.5f}")
    print(f"R-squared: {r2:.5f}")
    print(f"SMAPE: {smape_val:.5f}")
    print(f"Training Time: {training_time:.5f} seconds")
    print(f"CPU Usage: {psutil.cpu_percent()} MHz")
    print(f"Memory Used: {psutil.virtual_memory().used / 1024 / 1024} MB")
    print()

    return predictions, rmse, mae, r2, smape_val

# Create models
models = {
    'LSTM': create_lstm_model(128),
    'Stacked LSTM': create_stacked_lstm_model(128),
    'Bidirectional LSTM': create_bidirectional_lstm_model(128),
    'GRU': create_gru_model(128)
}

# Create lists to store evaluation metrics
best_model_predictions = {}
best_model_rmses = {}
best_model_maes = {}
best_model_r2s = {}
best_model_smapes = {}

# Train and evaluate each deep learning model
for model_name, model in models.items():
    predictions, rmse, mae, r2, smape_val = train_and_evaluate_model(model, model_name, X_train, y_train, X_test, y_test)

    best_model_predictions[model_name] = predictions
    best_model_rmses[model_name] = rmse
    best_model_maes[model_name] = mae
    best_model_r2s[model_name] = r2
    best_model_smapes[model_name] = smape_val

# Combine the predictions of the best models
combined_predictions = np.mean(list(best_model_predictions.values()), axis=0)
median_predictions = np.median(list(best_model_predictions.values()), axis=0)

# Compute evaluation metrics for the combined model
combined_mse = mean_squared_error(y_test, combined_predictions)
combined_rmse = np.sqrt(combined_mse)
combined_mae = mean_absolute_error(y_test, combined_predictions)
combined_r2 = r2_score(y_test, combined_predictions)
combined_smape = smape(y_test, combined_predictions)

# Compute evaluation metrics for the median model
median_mse = mean_squared_error(y_test, median_predictions)
median_rmse = np.sqrt(median_mse)
median_mae = mean_absolute_error(y_test, median_predictions)
median_r2 = r2_score(y_test, median_predictions)
median_smape = smape(y_test, median_predictions)

# Compute harmonic mean and geometric mean of the predictions
harmonic_mean_predictions = hmean(list(best_model_predictions.values()))
geometric_mean_predictions = gmean(list(best_model_predictions.values()))

# Compute evaluation metrics for harmonic mean predictions
harmonic_mean_mse = mean_squared_error(y_test, harmonic_mean_predictions)
harmonic_mean_rmse = np.sqrt(harmonic_mean_mse)
harmonic_mean_mae = mean_absolute_error(y_test, harmonic_mean_predictions)
harmonic_mean_r2 = r2_score(y_test, harmonic_mean_predictions)
harmonic_mean_smape = smape(y_test, harmonic_mean_predictions)

# Compute evaluation metrics for geometric mean predictions
geometric_mean_mse = mean_squared_error(y_test, geometric_mean_predictions)
geometric_mean_rmse = np.sqrt(geometric_mean_mse)
geometric_mean_mae = mean_absolute_error(y_test, geometric_mean_predictions)
geometric_mean_r2 = r2_score(y_test, geometric_mean_predictions)
geometric_mean_smape = smape(y_test, geometric_mean_predictions)

# Prepare the data for the evaluation metrics table
metrics_data = {
    'Model': list(models.keys()) + ['Combined Model', 'Median Model', 'Harmonic Mean Model', 'Geometric Mean Model'],
    'RMSE': list(best_model_rmses.values()) + [combined_rmse, median_rmse, harmonic_mean_rmse, geometric_mean_rmse],
    'MAE': list(best_model_maes.values()) + [combined_mae, median_mae, harmonic_mean_mae, geometric_mean_mae],
    'R-squared': list(best_model_r2s.values()) + [combined_r2, median_r2, harmonic_mean_r2, geometric_mean_r2],
    'SMAPE': list(best_model_smapes.values()) + [combined_smape, median_smape, harmonic_mean_smape, geometric_mean_smape]
}

# Create a DataFrame with the evaluation metrics
metrics_df = pd.DataFrame(metrics_data)

# Save the evaluation metrics to a CSV file
metrics_df.to_csv('deep_learning_model_metrics.csv', index=False)


3/3 [==============================] - 0s 2ms/step
Model: LSTM
RMSE: 4.41828
MAE: 3.71629
R-squared: 0.80264
SMAPE: 101.12625
Training Time: 35.26569 seconds
CPU Usage: 21.0 MHz
Memory Used: 14295.28125 MB

3/3 [==============================] - 1s 7ms/step
Model: Stacked LSTM
RMSE: 6.81194
MAE: 5.26734
R-squared: 0.53086
SMAPE: 101.59699
Training Time: 57.34942 seconds
CPU Usage: 49.0 MHz
Memory Used: 14362.44140625 MB

3/3 [==============================] - 1s 2ms/step
Model: Bidirectional LSTM
RMSE: 4.97584
MAE: 4.25540
R-squared: 0.74968
SMAPE: 100.50771
Training Time: 42.84408 seconds
CPU Usage: 53.2 MHz
Memory Used: 14378.984375 MB

3/3 [==============================] - 0s 8ms/step
Model: GRU
RMSE: 4.93938
MAE: 4.03686
R-squared: 0.75334
SMAPE: 100.92689
Training Time: 30.35164 seconds
CPU Usage: 34.3 MHz
Memory Used: 14461.7109375 MB



In [26]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Bidirectional, GRU
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from scipy.stats import hmean, gmean
import time
import psutil
import io
import requests

# Set a random seed for reproducibility
np.random.seed(42)

# Load the data
url = 'https://www.sidc.be/SILSO/INFO/snmstotcsv.php'
data = pd.read_csv(url, delimiter=';', header=None)
data.columns = ['Year', 'Month', 'Date', 'Monthly Mean Total Sunspot Number', 'Uncertainty', 'Observations', 'Definitive/Provisional']

# Select the 'Monthly Mean Total Sunspot Number' column as the target variable
target = data['Monthly Mean Total Sunspot Number'].values.reshape(-1, 1)

# Split the data into training and testing sets
train_size = 720  # Number of months for training
test_size = 72  # Number of months for testing

train_data = target[:train_size]
test_data = target[train_size:train_size+test_size]

# Split into input and output variables
X_train = train_data[:-1]
y_train = train_data[1:]
X_test = test_data[:-1]
y_test = test_data[1:]

# Create deep learning models
def create_lstm_model(units):
    model = Sequential()
    model.add(LSTM(units, input_shape=(1, X_train.shape[1])))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

def create_stacked_lstm_model(units):
    model = Sequential()
    model.add(LSTM(units, return_sequences=True, input_shape=(1, X_train.shape[1])))
    model.add(LSTM(units))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

def create_bidirectional_lstm_model(units):
    model = Sequential()
    model.add(Bidirectional(LSTM(units), input_shape=(1, X_train.shape[1])))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

def create_gru_model(units):
    model = Sequential()
    model.add(GRU(units, input_shape=(1, X_train.shape[1])))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

# Compute SMAPE (Symmetric Mean Absolute Percentage Error)
def smape(y_true, y_pred):
    return 200 * np.mean(np.abs(y_true - y_pred) / (np.abs(y_true) + np.abs(y_pred)))

# Train and evaluate models
def train_and_evaluate_model(model, model_name, X_train, y_train, X_test, y_test):
    start_time = time.time()
    history = model.fit(X_train.reshape((X_train.shape[0], 1, X_train.shape[1])),
                        y_train,
                        epochs=100,
                        batch_size=8,
                        verbose=0)
    training_time = time.time() - start_time

    predictions = model.predict(X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))).ravel()
    mse = mean_squared_error(y_test, predictions)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_test, predictions)
    r2 = r2_score(y_test, predictions)
    smape_val = smape(y_test, predictions)

    print(f"Model: {model_name}")
    print(f"RMSE: {rmse:.5f}")
    print(f"MAE: {mae:.5f}")
    print(f"R-squared: {r2:.5f}")
    print(f"SMAPE: {smape_val:.5f}")
    print(f"Training Time: {training_time:.5f} seconds")
    cpu_usage = psutil.cpu_percent()
    memory_usage = psutil.virtual_memory().used / 1024 / 1024
    print(f"CPU Usage: {cpu_usage:.1f} MHz")
    print(f"Memory Used: {memory_usage:.1f} MB")
    print()

    return predictions, rmse, mae, r2, smape_val, training_time, cpu_usage, memory_usage

# Create models
models = {
    'LSTM': create_lstm_model(128),
    'Stacked LSTM': create_stacked_lstm_model(128),
    'Bidirectional LSTM': create_bidirectional_lstm_model(128),
    'GRU': create_gru_model(128)
}

# Create lists to store evaluation metrics
best_model_predictions = {}
best_model_rmses = {}
best_model_maes = {}
best_model_r2s = {}
best_model_smapes = {}
training_times = {}
cpu_usages = {}
memory_usages = {}

# Train and evaluate each deep learning model
for model_name, model in models.items():
    predictions, rmse, mae, r2, smape_val, training_time, cpu_usage, memory_usage = train_and_evaluate_model(
        model, model_name, X_train, y_train, X_test, y_test)

    best_model_predictions[model_name] = predictions
    best_model_rmses[model_name] = rmse
    best_model_maes[model_name] = mae
    best_model_r2s[model_name] = r2
    best_model_smapes[model_name] = smape_val
    training_times[model_name] = training_time
    cpu_usages[model_name] = cpu_usage
    memory_usages[model_name] = memory_usage

# Combine the predictions of the best models
combined_predictions = np.mean(list(best_model_predictions.values()), axis=0)
median_predictions = np.median(list(best_model_predictions.values()), axis=0)

# Compute evaluation metrics for the combined model
combined_mse = mean_squared_error(y_test, combined_predictions)
combined_rmse = np.sqrt(combined_mse)
combined_mae = mean_absolute_error(y_test, combined_predictions)
combined_r2 = r2_score(y_test, combined_predictions)
combined_smape = smape(y_test, combined_predictions)

# Compute evaluation metrics for the median model
median_mse = mean_squared_error(y_test, median_predictions)
median_rmse = np.sqrt(median_mse)
median_mae = mean_absolute_error(y_test, median_predictions)
median_r2 = r2_score(y_test, median_predictions)
median_smape = smape(y_test, median_predictions)

# Compute harmonic mean and geometric mean of the predictions
harmonic_mean_predictions = hmean(list(best_model_predictions.values()))
geometric_mean_predictions = gmean(list(best_model_predictions.values()))

# Compute evaluation metrics for harmonic mean predictions
harmonic_mean_mse = mean_squared_error(y_test, harmonic_mean_predictions)
harmonic_mean_rmse = np.sqrt(harmonic_mean_mse)
harmonic_mean_mae = mean_absolute_error(y_test, harmonic_mean_predictions)
harmonic_mean_r2 = r2_score(y_test, harmonic_mean_predictions)
harmonic_mean_smape = smape(y_test, harmonic_mean_predictions)

# Compute evaluation metrics for geometric mean predictions
geometric_mean_mse = mean_squared_error(y_test, geometric_mean_predictions)
geometric_mean_rmse = np.sqrt(geometric_mean_mse)
geometric_mean_mae = mean_absolute_error(y_test, geometric_mean_predictions)
geometric_mean_r2 = r2_score(y_test, geometric_mean_predictions)
geometric_mean_smape = smape(y_test, geometric_mean_predictions)

# Prepare the data for the evaluation metrics table
metrics_data = {
    'Model': list(models.keys()) + ['Combined Model', 'Median Model', 'Harmonic Mean Model', 'Geometric Mean Model'],
    'RMSE': list(best_model_rmses.values()) + [combined_rmse, median_rmse, harmonic_mean_rmse, geometric_mean_rmse],
    'MAE': list(best_model_maes.values()) + [combined_mae, median_mae, harmonic_mean_mae, geometric_mean_mae],
    'R-squared': list(best_model_r2s.values()) + [combined_r2, median_r2, harmonic_mean_r2, geometric_mean_r2],
    'SMAPE': list(best_model_smapes.values()) + [combined_smape, median_smape, harmonic_mean_smape, geometric_mean_smape],
    'Training Time (seconds)': list(training_times.values()) + [None] * 4,
    'CPU Usage (MHz)': list(cpu_usages.values()) + [None] * 4,
    'Memory Used (MB)': list(memory_usages.values()) + [None] * 4
}

# Create a DataFrame with the evaluation metrics
metrics_df = pd.DataFrame(metrics_data)

# Save the evaluation metrics to a CSV file
metrics_df.to_csv('deep_learning_model_metrics2.csv', index=False)


3/3 [==============================] - 1s 3ms/step
Model: LSTM
RMSE: 4.66370
MAE: 3.86392
R-squared: 0.78010
SMAPE: 101.45635
Training Time: 35.35311 seconds
CPU Usage: 19.5 MHz
Memory Used: 14413.2 MB

3/3 [==============================] - 1s 3ms/step
Model: Stacked LSTM
RMSE: 5.99799
MAE: 4.84093
R-squared: 0.63627
SMAPE: 101.12073
Training Time: 60.15337 seconds
CPU Usage: 43.3 MHz
Memory Used: 14440.5 MB

3/3 [==============================] - 1s 5ms/step
Model: Bidirectional LSTM
RMSE: 5.14083
MAE: 4.13064
R-squared: 0.73280
SMAPE: 101.16717
Training Time: 43.53306 seconds
CPU Usage: 46.8 MHz
Memory Used: 14188.6 MB

3/3 [==============================] - 0s 3ms/step
Model: GRU
RMSE: 6.99919
MAE: 5.71987
R-squared: 0.50471
SMAPE: 100.52319
Training Time: 30.55079 seconds
CPU Usage: 31.0 MHz
Memory Used: 14173.6 MB



In [27]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.ensemble import AdaBoostRegressor, RandomForestRegressor, GradientBoostingRegressor, VotingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from scipy.stats import hmean, gmean
import time
import psutil
import io
import requests
import tensorflow as tf

# Set a random seed for reproducibility
np.random.seed(42)

# Load the data
# data = pd.read_csv('/content/SN_m_tot_V2.0.csv', delimiter=';', header=None)
# url = 'https://www.sidc.be/SILSO/INFO/snmtotcsv.php'
url = 'https://www.sidc.be/SILSO/INFO/snmstotcsv.php'
data = pd.read_csv(url, delimiter=';', header=None)
data.columns = ['Year', 'Month', 'Date', 'Monthly Mean Total Sunspot Number', 'Uncertainty', 'Observations', 'Definitive/Provisional']

# Select the 'Monthly Mean Total Sunspot Number' column as the target variable
target = data['Monthly Mean Total Sunspot Number'].values.reshape(-1, 1)

# Normalize the target variable
scaler = MinMaxScaler(feature_range=(0, 1))
target_scaled = scaler.fit_transform(target)

# Split the data into training and testing sets
train_size = 720  # Number of months for training
test_size = 72  # Number of months for testing

train_data = target_scaled[:train_size]
test_data = target_scaled[train_size:train_size+test_size]

# Split into input and output variables
X_train = train_data[:-1]
y_train = train_data[1:]
X_test = test_data[:-1]
y_test = test_data[1:]

if X_test.shape[0] == 0:
    X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, shuffle=False)

# Compute SMAPE on the original scale
def smape(y_true, y_pred):
    return 200 * np.mean(np.abs(y_true - y_pred) / (np.abs(y_true) + np.abs(y_pred)))

# Create deep learning models
def create_lstm_model(units):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.LSTM(units, input_shape=(1, X_train.shape[1])))
    model.add(tf.keras.layers.Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

def create_stacked_lstm_model(units):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.LSTM(units, input_shape=(1, X_train.shape[1]), return_sequences=True))
    model.add(tf.keras.layers.LSTM(units))
    model.add(tf.keras.layers.Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

def create_bidirectional_lstm_model(units):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(units), input_shape=(1, X_train.shape[1])))
    model.add(tf.keras.layers.Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

def create_gru_model(units):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.GRU(units, input_shape=(1, X_train.shape[1])))
    model.add(tf.keras.layers.Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

# Train and evaluate models
def train_and_evaluate_model(model, model_name, X_train, y_train, X_test, y_test):
    start_time = time.time()
    history = model.fit(X_train.reshape((X_train.shape[0], 1, X_train.shape[1])),
                        y_train,
                        epochs=100,
                        batch_size=8,
                        verbose=0)
    training_time = time.time() - start_time

    predictions = model.predict(X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))).ravel()
    predictions_original_scale = scaler.inverse_transform(predictions.reshape(-1, 1)).ravel()
    y_test_original_scale = scaler.inverse_transform(y_test).ravel()

    mse = mean_squared_error(y_test_original_scale, predictions_original_scale)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_test_original_scale, predictions_original_scale)
    r2 = r2_score(y_test_original_scale, predictions_original_scale)
    smape_val = smape(y_test_original_scale, predictions_original_scale)

    print(f"Model: {model_name}")
    print(f"RMSE: {rmse:.5f}")
    print(f"MAE: {mae:.5f}")
    print(f"R-squared: {r2:.5f}")
    print(f"SMAPE: {smape_val:.5f}")
    print(f"Training Time: {training_time:.5f} seconds")
    print(f"CPU Usage: {psutil.cpu_percent()} MHz")
    print(f"Memory Used: {psutil.virtual_memory().used / 1024 / 1024} MB")
    print()

    return predictions, rmse, mae, r2, smape_val

# Create models
models = {
    'LSTM': create_lstm_model(128),
    'Stacked LSTM': create_stacked_lstm_model(128),
    'Bidirectional LSTM': create_bidirectional_lstm_model(128),
    'GRU': create_gru_model(128)
}

# Create lists to store evaluation metrics
model_names = []
model_rmse = []
model_mae = []
model_r2 = []
model_smape = []
training_times = []
cpu_usages = []
memory_usages = []

# Train and evaluate each model
for model_name, model in models.items():
    model_names.append(model_name)
    predictions, rmse, mae, r2, smape_val = train_and_evaluate_model(model, model_name, X_train, y_train, X_test, y_test)
    model_rmse.append(rmse)
    model_mae.append(mae)
    model_r2.append(r2)
    model_smape.append(smape_val)

    # Append training time, CPU usage, and memory usage to lists
    training_times.append(time.time() - start_time)
    cpu_usages.append(psutil.cpu_percent())
    memory_usages.append(psutil.virtual_memory().used / 1024 / 1024)

# Combine the predictions of the best models
combined_predictions = np.mean([model.predict(X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))).ravel() for model in models.values()], axis=0)
median_predictions = np.median([model.predict(X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))).ravel() for model in models.values()], axis=0)

# Compute evaluation metrics for the combined model
combined_predictions_original_scale = scaler.inverse_transform(combined_predictions.reshape(-1, 1)).ravel()
combined_mse = mean_squared_error(y_test_original_scale, combined_predictions_original_scale)
combined_rmse = np.sqrt(combined_mse)
combined_mae = mean_absolute_error(y_test_original_scale, combined_predictions_original_scale)
combined_r2 = r2_score(y_test_original_scale, combined_predictions_original_scale)
combined_smape = smape(y_test_original_scale, combined_predictions_original_scale)

# Compute evaluation metrics for the median model
median_predictions_original_scale = scaler.inverse_transform(median_predictions.reshape(-1, 1)).ravel()
median_mse = mean_squared_error(y_test_original_scale, median_predictions_original_scale)
median_rmse = np.sqrt(median_mse)
median_mae = mean_absolute_error(y_test_original_scale, median_predictions_original_scale)
median_r2 = r2_score(y_test_original_scale, median_predictions_original_scale)
median_smape = smape(y_test_original_scale, median_predictions_original_scale)

# Compute harmonic mean and geometric mean of the predictions
best_model_predictions = [model.predict(X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))).ravel() for model in models.values()]
harmonic_mean_predictions = hmean(best_model_predictions)
geometric_mean_predictions = gmean(best_model_predictions)

# Compute evaluation metrics for harmonic mean predictions
harmonic_mean_predictions_original_scale = scaler.inverse_transform(harmonic_mean_predictions.reshape(-1, 1)).ravel()
harmonic_mean_mse = mean_squared_error(y_test_original_scale, harmonic_mean_predictions_original_scale)
harmonic_mean_rmse = np.sqrt(harmonic_mean_mse)
harmonic_mean_mae = mean_absolute_error(y_test_original_scale, harmonic_mean_predictions_original_scale)
harmonic_mean_r2 = r2_score(y_test_original_scale, harmonic_mean_predictions_original_scale)
harmonic_mean_smape = smape(y_test_original_scale, harmonic_mean_predictions_original_scale)

# Compute evaluation metrics for geometric mean predictions
geometric_mean_predictions_original_scale = scaler.inverse_transform(geometric_mean_predictions.reshape(-1, 1)).ravel()
geometric_mean_mse = mean_squared_error(y_test_original_scale, geometric_mean_predictions_original_scale)
geometric_mean_rmse = np.sqrt(geometric_mean_mse)
geometric_mean_mae = mean_absolute_error(y_test_original_scale, geometric_mean_predictions_original_scale)
geometric_mean_r2 = r2_score(y_test_original_scale, geometric_mean_predictions_original_scale)
geometric_mean_smape = smape(y_test_original_scale, geometric_mean_predictions_original_scale)

# Prepare the data for the evaluation metrics table
metrics_data = {
    'Model': model_names + ['Combined Model', 'Median Model', 'Harmonic Mean Model', 'Geometric Mean Model'],
    'RMSE': model_rmse + [combined_rmse, median_rmse, harmonic_mean_rmse, geometric_mean_rmse],
    'MAE': model_mae + [combined_mae, median_mae, harmonic_mean_mae, geometric_mean_mae],
    'R-squared': model_r2 + [combined_r2, median_r2, harmonic_mean_r2, geometric_mean_r2],
    'SMAPE': model_smape + [combined_smape, median_smape, harmonic_mean_smape, geometric_mean_smape],
    'Training Time (seconds)': training_times + [None] * 4,
    'CPU Usage (MHz)': cpu_usages + [None] * 4,
    'Memory Used (MB)': memory_usages + [None] * 4
}

# Save the evaluation metrics to a CSV file
metrics_df = pd.DataFrame(metrics_data)
metrics_df.to_csv('deep_learning_model_metrics3.csv', index=False)


3/3 [==============================] - 1s 8ms/step
Model: LSTM
RMSE: 1.11740
MAE: 0.85287
R-squared: 0.98738
SMAPE: 42.76431
Training Time: 32.64415 seconds
CPU Usage: 20.0 MHz
Memory Used: 14499.09765625 MB

3/3 [==============================] - 1s 7ms/step
Model: Stacked LSTM
RMSE: 6.28162
MAE: 6.16529
R-squared: 0.60106
SMAPE: 86.28563
Training Time: 57.05143 seconds
CPU Usage: 42.2 MHz
Memory Used: 14532.328125 MB

3/3 [==============================] - 1s 4ms/step
Model: Bidirectional LSTM
RMSE: 2.10087
MAE: 1.90499
R-squared: 0.95538
SMAPE: 59.81345
Training Time: 42.91935 seconds
CPU Usage: 45.9 MHz
Memory Used: 14587.0625 MB

3/3 [==============================] - 0s 9ms/step
Model: GRU
RMSE: 4.51402
MAE: 4.33633
R-squared: 0.79399
SMAPE: 77.31022
Training Time: 30.06283 seconds
CPU Usage: 39.9 MHz
Memory Used: 14545.7578125 MB

3/3 [==============================] - 0s 8ms/step


NameError: name 'y_test_original_scale' is not defined

In [23]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Bidirectional, GRU
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from scipy.stats import hmean, gmean
import time
import psutil
import io
import requests

# Set a random seed for reproducibility
np.random.seed(42)

# Load the data
url = 'https://www.sidc.be/SILSO/INFO/snmstotcsv.php'
data = pd.read_csv(url, delimiter=';', header=None)
data.columns = ['Year', 'Month', 'Date', 'Monthly Mean Total Sunspot Number', 'Uncertainty', 'Observations', 'Definitive/Provisional']

# Select the 'Monthly Mean Total Sunspot Number' column as the target variable
target = data['Monthly Mean Total Sunspot Number'].values.reshape(-1, 1)

# Split the data into training and testing sets
train_size = 720  # Number of months for training
test_size = 72  # Number of months for testing

train_data = target[:train_size]
test_data = target[train_size:train_size+test_size]

# Split into input and output variables
X_train = train_data[:-1]
y_train = train_data[1:]
X_test = test_data[:-1]
y_test = test_data[1:]

# Create deep learning models
def create_lstm_model(units):
    model = Sequential()
    model.add(LSTM(units, input_shape=(1, X_train.shape[1])))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

def create_bidirectional_lstm_model(units):
    model = Sequential()
    model.add(Bidirectional(LSTM(units), input_shape=(1, X_train.shape[1])))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

def create_gru_model(units):
    model = Sequential()
    model.add(GRU(units, input_shape=(1, X_train.shape[1])))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

# Compute SMAPE (Symmetric Mean Absolute Percentage Error)
def smape(y_true, y_pred):
    return 200 * np.mean(np.abs(y_true - y_pred) / (np.abs(y_true) + np.abs(y_pred)))

# Train and evaluate models
def train_and_evaluate_model(model, model_name, X_train, y_train, X_test, y_test):
    start_time = time.time()
    history = model.fit(X_train.reshape((X_train.shape[0], 1, X_train.shape[1])),
                        y_train,
                        epochs=100,
                        batch_size=8,
                        verbose=0)
    training_time = time.time() - start_time

    predictions = model.predict(X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))).ravel()
    mse = mean_squared_error(y_test, predictions)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_test, predictions)
    r2 = r2_score(y_test, predictions)
    smape_val = smape(y_test, predictions)

    print(f"Model: {model_name}")
    print(f"RMSE: {rmse:.5f}")
    print(f"MAE: {mae:.5f}")
    print(f"R-squared: {r2:.5f}")
    print(f"SMAPE: {smape_val:.5f}")
    print(f"Training Time: {training_time:.5f} seconds")
    print(f"CPU Usage: {psutil.cpu_percent()} MHz")
    print(f"Memory Used: {psutil.virtual_memory().used / 1024 / 1024} MB")
    print()

    return predictions, rmse, mae, r2, smape_val

# Create models
models = {
    'LSTM': create_lstm_model(128),
    'Bidirectional LSTM': create_bidirectional_lstm_model(128),
    'GRU': create_gru_model(128)
}

# Create lists to store evaluation metrics
best_model_predictions = {}
best_model_rmses = {}
best_model_maes = {}
best_model_r2s = {}
best_model_smapes = {}

# Train and evaluate each deep learning model
for model_name, model in models.items():
    predictions, rmse, mae, r2, smape_val = train_and_evaluate_model(model, model_name, X_train, y_train, X_test, y_test)

    best_model_predictions[model_name] = predictions
    best_model_rmses[model_name] = rmse
    best_model_maes[model_name] = mae
    best_model_r2s[model_name] = r2
    best_model_smapes[model_name] = smape_val

# Combine the predictions of the best models
combined_predictions = np.mean(list(best_model_predictions.values()), axis=0)
median_predictions = np.median(list(best_model_predictions.values()), axis=0)

# Compute evaluation metrics for the combined model
combined_mse = mean_squared_error(y_test, combined_predictions)
combined_rmse = np.sqrt(combined_mse)
combined_mae = mean_absolute_error(y_test, combined_predictions)
combined_r2 = r2_score(y_test, combined_predictions)
combined_smape = smape(y_test, combined_predictions)

# Compute evaluation metrics for the median model
median_mse = mean_squared_error(y_test, median_predictions)
median_rmse = np.sqrt(median_mse)
median_mae = mean_absolute_error(y_test, median_predictions)
median_r2 = r2_score(y_test, median_predictions)
median_smape = smape(y_test, median_predictions)

# Compute harmonic mean and geometric mean of the predictions
harmonic_mean_predictions = hmean(list(best_model_predictions.values()))
geometric_mean_predictions = gmean(list(best_model_predictions.values()))

# Compute evaluation metrics for harmonic mean predictions
harmonic_mean_mse = mean_squared_error(y_test, harmonic_mean_predictions)
harmonic_mean_rmse = np.sqrt(harmonic_mean_mse)
harmonic_mean_mae = mean_absolute_error(y_test, harmonic_mean_predictions)
harmonic_mean_r2 = r2_score(y_test, harmonic_mean_predictions)
harmonic_mean_smape = smape(y_test, harmonic_mean_predictions)

# Compute evaluation metrics for geometric mean predictions
geometric_mean_mse = mean_squared_error(y_test, geometric_mean_predictions)
geometric_mean_rmse = np.sqrt(geometric_mean_mse)
geometric_mean_mae = mean_absolute_error(y_test, geometric_mean_predictions)
geometric_mean_r2 = r2_score(y_test, geometric_mean_predictions)
geometric_mean_smape = smape(y_test, geometric_mean_predictions)

# Prepare the data for the evaluation metrics table
metrics_data = {
    'Model': list(models.keys()) + ['Combined Model', 'Median Model', 'Harmonic Mean Model', 'Geometric Mean Model'],
    'RMSE': list(best_model_rmses.values()) + [combined_rmse, median_rmse, harmonic_mean_rmse, geometric_mean_rmse],
    'MAE': list(best_model_maes.values()) + [combined_mae, median_mae, harmonic_mean_mae, geometric_mean_mae],
    'R-squared': list(best_model_r2s.values()) + [combined_r2, median_r2, harmonic_mean_r2, geometric_mean_r2],
    'SMAPE': list(best_model_smapes.values()) + [combined_smape, median_smape, harmonic_mean_smape, geometric_mean_smape]
}

# Create a DataFrame with the evaluation metrics
metrics_df = pd.DataFrame(metrics_data)

# Save the evaluation metrics to a CSV file
metrics_df.to_csv('deep_learning_model_metrics.csv', index=False)


3/3 [==============================] - 0s 0s/step
Model: LSTM
RMSE: 4.56139
MAE: 3.89048
R-squared: 0.78964
SMAPE: 100.92687
Training Time: 34.64167 seconds
CPU Usage: 21.1 MHz
Memory Used: 14526.9140625 MB

3/3 [==============================] - 1s 8ms/step
Model: Bidirectional LSTM
RMSE: 5.52531
MAE: 4.52270
R-squared: 0.69134
SMAPE: 100.65072
Training Time: 44.65468 seconds
CPU Usage: 41.8 MHz
Memory Used: 14445.921875 MB

3/3 [==============================] - 0s 3ms/step
Model: GRU
RMSE: 5.66491
MAE: 4.51402
R-squared: 0.67555
SMAPE: 101.46372
Training Time: 33.19287 seconds
CPU Usage: 38.6 MHz
Memory Used: 14462.1328125 MB

